# Ray Optics and Glass Equations

In [2]:
import numpy as np
from numpy import sin, cos, pi, sqrt

import matplotlib.pyplot as plt

# enables seeing multiple outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Lensmaker's equation and spot size

In [3]:
# parameters (in meters)

R1 = 0.1           # radii
R2 = -0.1
n = 1.5            # refractive index
lambda_ = 550e-9   # wavelength of light used
CA = 0.05          # clear aperture 
t = 0.01           # lens thickness

In [4]:
# equations

phi = (n - 1) * (1/R1 - 1/R2 + (n-1)*t/(n*R1*R2))
f = 1/phi
f_num = f/CA
S = 2.44 * lambda_ * f_num

In [5]:
print('Optical power: {0:1.2f} diopters'.format(phi))
print('Focal length: {0:1.2f} m'.format(f))
print('f-number: {0:1.2f} m'.format(f_num))
print('Spot diameter: {0:1.2f} um'.format(S*1e6))

Optical power: 9.83 diopters
Focal length: 0.10 m
f-number: 2.03 m
Spot diameter: 2.73 um


## Matrix method for ray tracing
- **Example 6.2**: Find the focal length of a spherical lens with a radius of 2cm and an refractive index of 1.3.

In [6]:
# parameters

n_glass = 1.3
n_air = 1
lens_diameter = 4
lens_roc = lens_diameter/2

In [7]:
# translation and refraction matrices

P1 = (n_air - n_glass) / (-lens_roc)    # 1st surface lens power
P2 = (n_glass - n_air) / (lens_roc)     # 2nd surface lens power
L_n = lens_diameter / n_glass

In [8]:
test = 10
f = 0

while abs(test) > 0.0005:   # force M[1,1] to 0, as at the focal point the height is 0
    f +=  0.0001

    # translation matrices
    TM1 = np.array([[1, 0], 
                    [f, 1]], dtype=np.float64)
    TM2 = np.array([[1, 0], 
                    [L_n, 1]], dtype=np.float64)

    # refraction matrices
    RM1 = np.array([[1, -P1], 
                    [0, 1]])
    RM2 = np.array([[1, -P2], 
                    [0, 1]])

    # calculate ray-transfer matrix
    Mlens = RM2.dot(TM2).dot(RM1)
    M = TM1.dot(Mlens)

    A = M[0,0]
    B = M[0,1]
    C = M[1,0]
    D = M[1,1]

    test = D

In [9]:
print(Mlens, '\n')
print(M, '\n')
print(f)

[[ 0.53846154 -0.23076923]
 [ 3.07692308  0.53846154]] 

[[ 5.38461538e-01 -2.30769231e-01]
 [ 4.33218462e+00  4.92307692e-04]] 

2.331200000000495


## Ray tracing through multiple elements
- **Example 6.3**: Develop the ray-transfer matrix for a two-element inverting telescope.

In [10]:
# parameters

fe = 1
fo = 100
y0 = 1
theta0 = .0

In [11]:
# configure translation and refraction matrices

X = fo + fe
Po = 1/fo
Pe = 1/fe
TM1 = np.array([[1, 0],
                [X, 1]])
RM1 = np.array([[1, -Pe],
                [0, 1]])
RM2 = np.array([[1, -Po],
                [0, 1]])

In [12]:
# calaulate ray-transfer matrix

M = RM2.dot(TM1).dot(RM1)

A = M[0,0]
B = M[0,1]
C = M[1,0]
D = M[1,1]

In [13]:
# exit ray
initial_ray = np.array([theta0, y0])
N = initial_ray.dot(M)

In [18]:
print('Ray-transfer matrix:\n {}'.format(M))
print()
print('Final ray:\n {}'.format(N))

Ray-transfer matrix:
 [[-1.00000000e-02  2.08166817e-17]
 [ 1.01000000e+02 -1.00000000e+02]]

Final ray:
 [ 101. -100.]
